In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tfc
import numpy as np
import seaborn as sns

from tensorflow import keras
from sklearn.metrics import confusion_matrix

In [ ]:
tf.keras.backend.clear_session()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
try: sess.close() 
except: pass

tfc.enable_eager_execution()
print(f"eagely execution: {tf.executing_eagerly()}")

gpu_options= tfc.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tfc.InteractiveSession(config=tfc.ConfigProto(gpu_options=gpu_options))

## Constants

In [ ]:
CHANNELS = 1
INPUT_SHAPE = [218,145,CHANNELS]

BATCH_SIZE = 64

SOURCE = "train_images/"
RES = f"train_images_res_{INPUT_SHAPE[0]}x{INPUT_SHAPE[1]}/"

MODELS = [] #-- insert models

## Input Pipeline

In [ ]:
@tf.function
def process_tuple(image,label):
    img = tf.io.read_file(image)
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float16)
    return img, label

In [ ]:
test_even_df = pd.read_pickle("test_ds.pickle").sort_index()

In [ ]:
test_label = list(test_even_df["species"].values)
test_img = list(map(RES.__add__,test_even_df["image"]))
test_ds = tf.data.Dataset.from_tensor_slices((test_img,test_label)).shuffle(1000, seed = 1337).map(process_tuple,num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)

In [ ]:
test_batched = test_ds.batch(batch_size=BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

## Evaluate models

In [ ]:
def make_conf_matrix(y_act,y_pred,model):
    cm=confusion_matrix(y_act,y_pred)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100

    plt.figure(figsize=(11, 11))

    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            else:
                annot[i, j] = '%d' % (c)

    cm=confusion_matrix(y_act,y_pred,normalize='true')            
    cm = pd.DataFrame(cm, index=names, columns=names)

    ax = sns.heatmap(cm, annot=annot, fmt='', cbar=True, cmap="Blues")

    ax.set_title('Confusion Matrix\n\n')
    ax.set_xlabel('\nPredicted Species')
    ax.set_ylabel('Actual Species')

    ax.xaxis.set_ticklabels(names,rotation = 90)
    ax.yaxis.set_ticklabels(names, rotation = 360)

    plt.savefig(model)

In [ ]:
y_act = np.array(test_label)

metrics = []
for model in MODELS:
    cm = keras.models.load_model(model)
    pr = cm.predict(test_batched)
    
    y_pred = pr.argmax(axis=1)
    make_conf_matrix(y_act,y_pred,model)
    
    cval = current_model.evaluate(test_batched, verbose = 0)
    print(f"{model} test loss, test acc: {cval}")
    
    
    test_error_rate = round((1-cval[1])*100,2)
    metrics.append([model, cval,test_error_rate])